In [9]:
!pip install pyspark
!pip install datetime-month

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('prac05').getOrCreate()
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for datetime-month: filename=datetime_month-1.0.4-py2.py3-none-any.whl size=8032 sha256=81275a706d9c41ecec77b098bc2f9655039b340ecc0064486fba875b482272d8
  Stored in directory: /root/.cache/pip/wheels/8b/d7/84/3580b0314aadcc5e2d3e922dac9e86f764e79ee225f0f4de88
Successfully built datetime-month


In [4]:
text_rdd = spark.sparkContext.textFile("/content/drive/MyDrive/Colab Notebooks/spark/weather_data.txt")

map_rdd = text_rdd.map(lambda line: line.split(' ')).toDF([ 
    'WBANNO','LST_DATE','CRX_VN','LONGITUDE','LATITUDE','T_DAILY_MAX','T_DAILY_MIN', 'T_DAILY_MEAN'
])

map_rdd.show(2)

+------+--------+------+---------+--------+-----------+-----------+------------+---+---+----+---+---+---+---+-----+----+----+-----+-----+-------+-------+-------+---+---+-------+-------+-------+
|WBANNO|LST_DATE|CRX_VN|LONGITUDE|LATITUDE|T_DAILY_MAX|T_DAILY_MIN|T_DAILY_MEAN| _9|_10| _11|_12|_13|_14|_15|  _16| _17| _18|  _19|  _20|    _21|    _22|    _23|_24|_25|    _26|    _27|    _28|
+------+--------+------+---------+--------+-----------+-----------+------------+---+---+----+---+---+---+---+-----+----+----+-----+-----+-------+-------+-------+---+---+-------+-------+-------+
| 23907|20150101| 2.423|   -98.08|   30.62|        2.2|       -0.6|         0.8|0.9|6.2|1.47|  C|3.7|1.1|2.5| 99.9|85.4|97.2|0.369|0.308|-99.000|-99.000|-99.000|7.0|8.1|-9999.0|-9999.0|-9999.0|
| 23907|20150102| 2.423|   -98.08|   30.62|        3.5|        1.3|         2.4|2.2|9.0|1.43|  C|4.9|2.3|3.1|100.0|98.8|99.8|0.391|0.327|-99.000|-99.000|-99.000|7.1|7.9|-9999.0|-9999.0|-9999.0|
+------+--------+------+------

In [5]:
map_rdd.select( max('T_DAILY_MAX'), min('T_DAILY_MIN')).show()

+----------------+----------------+
|max(T_DAILY_MAX)|min(T_DAILY_MIN)|
+----------------+----------------+
|             9.4|            -0.2|
+----------------+----------------+



In [8]:
map_rdd.select( substring('LST_DATE', 5, 2).alias('monthVal'),'T_DAILY_MAX', 'T_DAILY_MIN')\
        .groupby('monthVal')\
        .agg(max('T_DAILY_MAX'), min('T_DAILY_MIN'))\
        .show()

+--------+----------------+----------------+
|monthVal|max(T_DAILY_MAX)|min(T_DAILY_MIN)|
+--------+----------------+----------------+
|      01|             9.4|            -0.6|
|      02|             9.4|            -0.4|
|      03|             4.9|            -0.2|
|      04|            30.8|            10.7|
|      05|            31.1|            14.3|
|      06|            33.6|             0.0|
|      07|            36.0|            19.8|
+--------+----------------+----------------+



### with sql query

In [13]:
map_rdd.select( substring('LST_DATE', 5, 2).alias('monthVal'),'T_DAILY_MAX', 'T_DAILY_MIN').registerTempTable("weather")

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [14]:
query = ''' 
select monthVal, max(T_DAILY_MAX), min(T_DAILY_MIN) from weather 
group by monthVal
'''

spark.sql(query).show()

+--------+----------------+----------------+
|monthVal|max(T_DAILY_MAX)|min(T_DAILY_MIN)|
+--------+----------------+----------------+
|      01|             9.4|            -0.6|
|      02|             9.4|            -0.4|
|      03|             4.9|            -0.2|
|      04|            30.8|            10.7|
|      05|            31.1|            14.3|
|      06|            33.6|             0.0|
|      07|            36.0|            19.8|
+--------+----------------+----------------+

